In [1]:
import pandas as pd
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [2]:
fields = ["Date_HPSC", "TotalLabs", "Test24", "PRate"]
cov_nums = pd.read_csv("LaboratoryLocalTimeSeriesHistoricView.csv", usecols=fields)
cov_nums.head()

,Date_HPSC,TotalLabs,PRate,Test24
0,2020/03/18 11:00:00+00,6457,6.1,0
1,2020/03/19 11:00:00+00,7403,6.3,946
2,2020/03/20 11:00:00+00,8872,6.4,1469
3,2020/03/21 11:00:00+00,10436,6.0,1564
4,2020/03/22 11:00:00+00,11414,6.5,978


In [3]:
def positive(row):
    pos = (row['Test24'] / 100) * row['PRate']
    return round(pos)

cov_nums['daily_pos'] = cov_nums.apply(positive, axis=1)
cov_nums = cov_nums.iloc[1:]
cov_nums.tail()

,Date_HPSC,TotalLabs,PRate,Test24,daily_pos
389,2021/04/11 11:00:00+00,4184407,5.9,14355,847
390,2021/04/12 11:00:00+00,4196325,5.9,11918,703
391,2021/04/13 11:00:00+00,4209975,5.9,13650,805
392,2021/04/14 11:00:00+00,4226847,5.9,16872,995
393,2021/04/15 11:00:00+00,4241709,5.9,14862,877


In [4]:
test_info = ["Date_HPSC", "Test24"]
test_df = cov_nums[test_info]
test_df = pd.DataFrame(test_df)
test_df['Date_HPSC'] = pd.to_datetime(test_df['Date_HPSC']).dt.date
test_df.head()

,Date_HPSC,Test24
1,2020-03-19,946
2,2020-03-20,1469
3,2020-03-21,1564
4,2020-03-22,978
5,2020-03-23,2581


In [5]:
pos = ['Date_HPSC','Test24','daily_pos','PRate']
pos_df = cov_nums[pos]
pos_df = pd.DataFrame(pos_df)
pos_df['Date_HPSC'] = pd.to_datetime(pos_df['Date_HPSC']).dt.date

In [6]:
def seq(row):
    length = (int(row['Test24']) - int(row['daily_pos']))
    daily_pos = int(row['daily_pos'])
    init_seq = [0] * length
    init_pos = [1] * daily_pos
    seq_list = init_seq + init_pos
    for i in range(len(seq_list)-1, 0, -1):
        j = random.randint(0, i+1)
        seq_list[i], seq_list[j] = seq_list[j], seq_list[i]
    return list(seq_list)

pos_df['seq_list'] = pos_df.apply(seq, axis=1)
pos_df = pd.DataFrame(pos_df)


In [7]:
def hasPositive(block):
    for node in block:
        if node == 1:
            return True
    return False

def splitBlock(block):
    halfBlock = len(block) // 2
    return block[:halfBlock], block[halfBlock:]

def binSplit(data, count):
    count += 1
    #print("Data:", data)
    if len(data) > 1 and hasPositive(data):     # Branch is complete
        left, right = splitBlock(data)      # Split the block
       # print("Left:", left, "\tRight:", right)
        if len(left) <= 1 or not hasPositive(left):     # Left branch is complete
        #    print("Left branch is complete")
            return binSplit(right, count)
        result, count = binSplit(left, count)
      #  print("Result:", result)
        if result is True:
       #     print("Result was True")
            return binSplit(right, count)
      #  print("Adding result to right and splitting")
        return binSplit((result + right), count)
    else:
     #   print("Returning True")
        return True, count

def runtime(row):
    answer = binSplit(row, 0)[1]
    return answer

pos_df['pool_test'] = pos_df['seq_list'].apply(runtime)
pos_df['pool_test'].head()
pos_df = pos_df.drop(['seq_list'], axis=1)

df = pos_df
df.head()

,Date_HPSC,Test24,daily_pos,PRate,pool_test
1,2020-03-19,946,60,6.3,377
2,2020-03-20,1469,94,6.4,587
3,2020-03-21,1564,94,6.0,591
4,2020-03-22,978,64,6.5,390
5,2020-03-23,2581,157,6.1,990


In [8]:
pred_pool = ['Date_HPSC', 'pool_test']
pred_pool = df[pred_pool]
pred_pool.rename(columns={'Date_HPSC':'ds',
                          'pool_test':'y'},
                 inplace=True)


C:\Users\80317960\Anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [10]:
m = Prophet().fit(pred_pool)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [12]:
future = m.make_future_dataframe(periods=30)
future.head()

,ds
0,2020-03-19
1,2020-03-20
2,2020-03-21
3,2020-03-22
4,2020-03-23


In [13]:
forecast = m.predict(future)
forecast.rename(columns={'ds':'Date_HPSC',
                         'trend': 'pred_pool'},
                inplace=True)
forecast = forecast.set_index('Date_HPSC')
forecast.head()


,pred_pool,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
Date_HPSC,,,,,,,,,,,,,,,
2020-03-19,2168.709992,756.137421,4378.261774,2168.709992,2168.709992,336.702071,336.702071,336.702071,336.702071,336.702071,336.702071,0.0,0.0,0.0,2505.412062
2020-03-20,2169.552260,952.356989,4523.888506,2169.552260,2169.552260,521.439607,521.439607,521.439607,521.439607,521.439607,521.439607,0.0,0.0,0.0,2690.991867
2020-03-21,2170.394528,625.791889,4288.471332,2170.394528,2170.394528,355.742636,355.742636,355.742636,355.742636,355.742636,355.742636,0.0,0.0,0.0,2526.137164
2020-03-22,2171.236796,245.887405,3823.225621,2171.236796,2171.236796,-186.972122,-186.972122,-186.972122,-186.972122,-186.972122,-186.972122,0.0,0.0,0.0,1984.264674
2020-03-23,2172.079064,-404.738596,3387.757270,2172.079064,2172.079064,-623.842894,-623.842894,-623.842894,-623.842894,-623.842894,-623.842894,0.0,0.0,0.0,1548.236170


In [15]:
forecast.to_csv('out_data/forecast.csv')
df.to_csv('out_data/pool_test.csv')